In [5]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torch.optim as optim
import torch.nn as nn

from qibo import Circuit, gates, hamiltonians, set_backend, construct_backend

from qiboml.models.encoding import PhaseEncoding
from qiboml.models.decoding import Expectation
from qiboml.interfaces.pytorch import QuantumModel
from qiboml.operations.differentiation import PSR, Jax

### Construct model

In [11]:
set_backend("qiboml", platform="pytorch")

nqubits = 2
nlayers = 3

# Encoding layer
encoding_circ = PhaseEncoding(
    nqubits=nqubits, 
    encoding_gate=gates.RX
)

# Trainable layer
def trainable_circuit(entanglement=True):
    trainable_circ  = Circuit(nqubits)
    for q in range(nqubits):
        trainable_circ.add(gates.RY(q=q, theta=np.random.randn()))
        trainable_circ.add(gates.RZ(q=q, theta=np.random.randn()))
    if nqubits > 1 and entanglement:
        [trainable_circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    return trainable_circ

[Qibo 0.2.17|INFO|2025-03-27 10:52:51]: Using qiboml (pytorch) backend on cpu


### Test function

In [12]:
def compute_gradient_wrt_x(
    execution_backend,
    differentiation_rule,
    trainable_circ_list,
):
    """Mount the model, execute it and compute gradient w.r.t. input data."""
    # Decoding layer
    decoding_circ = Expectation(
        nqubits=nqubits, 
        backend=execution_backend
    )

    structure = []
    for i, circ in enumerate(trainable_circ_list):
        structure.extend([
            encoding_circ, circ
        ])

    # The whole model
    model = QuantumModel(
        circuit_structure=structure,
        decoding=decoding_circ,
        differentiation=differentiation_rule,
    )

    data = torch.tensor([0.3, 0.9], requires_grad=True)
    
    output = model(data)
    loss = output
    loss.backward()

    print(f"Grad via {differentiation_rule} on {execution_backend} backend:", data.grad)

In [14]:
# Construct a list of trainable layers so that the params are the same in all tests
trainable_circ_list = []
for _ in range(nlayers):
    trainable_circ_list.append(trainable_circuit(entanglement=True))

### Execute on different backends and with different diffrules

In [15]:
compute_gradient_wrt_x(
    execution_backend=construct_backend("numpy"),
    differentiation_rule=PSR(),
    trainable_circ_list=trainable_circ_list,
)
compute_gradient_wrt_x(
    execution_backend=construct_backend("qiboml", platform="pytorch"),
    differentiation_rule=None,
    trainable_circ_list=trainable_circ_list,
)
compute_gradient_wrt_x(
    execution_backend=construct_backend("numpy"),
    differentiation_rule=Jax(),
    trainable_circ_list=trainable_circ_list,
)

Grad via PSR() on numpy backend: tensor([0.6515, 0.3940])
Grad via None on qiboml (pytorch) backend: tensor([0.6515, 0.3940])
Grad via Jax() on numpy backend: tensor([0.6515, 0.3940])
